In [56]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [57]:
from warnings import filterwarnings
filterwarnings('ignore')

KNN

In [58]:
df = pd.read_csv("C:/Users/safye/Desktop/makine_ogrenmesi/dogRegMod/datalarr/archive/Hitters.csv")
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [59]:
X_train.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
183,328.0,91.0,12.0,51.0,43.0,33.0,2.0,342.0,94.0,12.0,51.0,44.0,33.0,145.0,59.0,8.0,True,False,True
229,514.0,144.0,0.0,67.0,54.0,79.0,9.0,4739.0,1169.0,13.0,583.0,374.0,528.0,229.0,453.0,15.0,True,False,True
286,593.0,152.0,23.0,69.0,75.0,53.0,6.0,2765.0,686.0,133.0,369.0,384.0,321.0,315.0,10.0,6.0,False,True,False
102,233.0,49.0,2.0,41.0,23.0,18.0,8.0,1350.0,336.0,7.0,166.0,122.0,106.0,102.0,132.0,10.0,False,False,False
153,341.0,95.0,6.0,48.0,42.0,20.0,10.0,2964.0,808.0,81.0,379.0,428.0,221.0,158.0,4.0,5.0,True,True,True


Model ve Tahmin

In [60]:
knn_model = KNeighborsRegressor().fit(X_train, y_train)

In [61]:
knn_model

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [62]:
knn_model.n_neighbors

5

In [63]:
knn_model.metric

'minkowski'

In [64]:
dir(knn_model)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__sklearn_tags__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_algorithm_metric',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_fit',
 '_fit_X',
 '_fit_method',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_params_html',
 '_html_repr',
 '_kneighbors_reduce_func',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_tree',
 '_

In [65]:
knn_model.predict(X_test)[0:5]

array([ 510.3334,  808.3334,  772.5   ,  125.5   , 1005.    ])

In [66]:
y_pred = knn_model.predict(X_test)

In [67]:
np.sqrt(mean_squared_error(y_test, y_pred))

np.float64(426.6570764525201)

Model Tuning

In [68]:
RMSE = []

for k in range(10):
    k = k + 1
    knn_model = KNeighborsRegressor(n_neighbors=k).fit(X_train, y_train)
    y_pred = knn_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    RMSE.append(rmse)
    print("k", k, "değeri için tahmin değeri ", rmse, "dir")

k 1 değeri için tahmin değeri  455.03925390751965 dir
k 2 değeri için tahmin değeri  415.99629571490965 dir
k 3 değeri için tahmin değeri  420.6765370082348 dir
k 4 değeri için tahmin değeri  428.8564674588792 dir
k 5 değeri için tahmin değeri  426.6570764525201 dir
k 6 değeri için tahmin değeri  423.5071669008732 dir
k 7 değeri için tahmin değeri  414.9361222421057 dir
k 8 değeri için tahmin değeri  413.7094731463598 dir
k 9 değeri için tahmin değeri  417.84419990871265 dir
k 10 değeri için tahmin değeri  421.6252180741266 dir


In [69]:
RMSE

[np.float64(455.03925390751965),
 np.float64(415.99629571490965),
 np.float64(420.6765370082348),
 np.float64(428.8564674588792),
 np.float64(426.6570764525201),
 np.float64(423.5071669008732),
 np.float64(414.9361222421057),
 np.float64(413.7094731463598),
 np.float64(417.84419990871265),
 np.float64(421.6252180741266)]

Grid Search CV

In [70]:
knn_params = {"n_neighbors": np.arange(1, 30 , 1)}

In [71]:
knn = KNeighborsRegressor()

In [72]:
knn_cv_model = GridSearchCV(knn, knn_params, cv=10).fit(X_train, y_train)

In [73]:
knn_cv_model.best_params_

{'n_neighbors': np.int64(8)}

Final Model

In [74]:
knn_tuned = KNeighborsRegressor(n_neighbors = knn_cv_model.best_params_["n_neighbors"]).fit(X_train, y_train)

In [76]:
y_pred = knn_tuned.predict(X_test)

In [77]:
np.sqrt(mean_squared_error(y_test, y_pred))

np.float64(413.7094731463598)